<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Attributions_Longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook adapts the [Captum tutorial for question answering](https://captum.ai/tutorials/Bert_SQUAD_Interpret) and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get token attributions for the examples of your choice, or the entire dataset if needed. By doing so, we can visualize which tokens have the most influence in the model's prediction, and find out the k tokens with the most influence at helping the model predict correctly as well as incorrectly.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 59.5 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 101 kB 2.9 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 5.3 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 141 kB 44.0 MB/s 
     |████████████████████████████████| 212 kB 41.0 MB/s 
     |████████████████████████████████| 115 kB 29.9 MB/s 
     |████████████████████████████████| 127 kB 55.3 MB/s 


In [ ]:
pip install nltk --quiet

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

Here we are importing the model and tokenizer and letting the model use our GPU to run. Please change model path, and tokenizer to whichever one you wish to use.

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-news-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at danielhou13/longformer-finetuned_papers_v2 were not used when initializing LongformerForSequenceClassification: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Create functions that give us the input ids and the position ids for the text we want to examine along with the baselines for integrated gradients.

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the text

Please adjust the max_length accordingly for your project. The length should be the length you desire subtracted by 2 (as we are adding the CLS token at the beginning and the seperator token at the end.

In [ ]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    position_ids = position_ids[:, :seq_length]
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

### Import Dataset

Here we import the papers dataset

In [ ]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["validation"]

Downloading:   0%|          | 0.00/739 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-144b958ac1a53abb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds = load_dataset("danielhou13/cogs402dataset2")["validation"]

## Getting the Attributions

A custom forward function that returns the softmaxed logits, which are the class probabilities that the model uses for prediction.

In [ ]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

A helper function to summarize attributions for each word token in the sequence. The attribution output has a shape of (seq_len, model_embedding_size) so this function summarizes the output to an array of shape (seq_len).

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

Perform Layer Integrated Gradients using the longformer's embeddings.

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further examine the attributions scores for each example. We remove the whitespace character than the Longformer Tokenizer applies. More information about the integrated gradients function can be found [here](https://captum.ai/api/layer.html#layer-integrated-gradients).

In [ ]:
vis_data_records = []
all_attributions = {}
all_tokens = {}
all_deltas = {}

In [ ]:
# Takes in dataset and example number
def get_token_attributions(dataset, example):
  text = dataset['text'][example]
  label = dataset['labels'][example]

  # get the inputs, position ids, attention mask, and the baselines
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  #get the tokens
  indices = input_ids[0].detach().tolist()
  all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)
  all_tokens[str(example)] = all_tokens_curr

  #perform integrated gradients
  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=1,
                                    n_steps=500,
                                    internal_batch_size = 2)

  # We want one value for every token.
  attributions_sum = summarize_attributions(attributions)

  # store the values in our dictionary
  all_attributions[str(example)] = attributions_sum
  all_deltas[str(example)] = attributions_sum

  # get the score for our visualization
  score = predict(input_ids, position_ids, attention_mask)

  # storing couple samples in an array for visualization purposes
  # requires array of attributions, prediction score, predicted class, true class 
  # the label you want your attributions to associate positive with, the attribution score
  # the tokens, and the delta if you have it.
  vis_data_records.append(viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        str(1),
                        attributions_sum.sum(),       
                        all_tokens_curr,
                        delta)
  )

Here we are taking some examples from the Papers datasets.

In [ ]:
# get_token_attributions(cogs402_ds, 976)
get_token_attributions(cogs402_ds, 891)
# get_token_attributions(cogs402_ds, 589)
# get_token_attributions(cogs402_ds, 605)
# get_token_attributions(cogs402_ds, 148)

Here we are taking some examples from the News datasets.

In [ ]:
# get_token_attributions(cogs402_ds, 102)
# get_token_attributions(cogs402_ds, 1168)
# # get_token_attributions(cogs402_ds, 2307)
# # get_token_attributions(cogs402_ds, 2359)

This function allows us to display our attributions in a manner that is easy to read. We can see the attributions of the word overlayed on top of their respective token. The green colour represents positive attributions (i.e. the model is attributing this token to influential for predicting the positive class) while the red colour represents negative attributions. 

In [ ]:
# # storing couple samples in an array for visualization purposes
# score_vis = viz.VisualizationDataRecord(
#                         attributions_sum,
#                         torch.softmax(score, dim = 1).max(),
#                         torch.argmax(torch.softmax(score, dim = 1)),
#                         label,
#                         str(1),
#                         attributions_sum.sum(),       
#                         all_tokens,
#                         delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
_ = viz.visualize_text(vis_data_records)

 Visualization For Score 


## Further Examination of the Attributions

Next we might want to look in-depth about the attribution scores for each token of an example. We saved the attributions for the examples we looked at above, so we can easily retrieve the attributions. We also grab the examples because we want to know what tokens the attributions are associated with.

Both lists are of shape: (seq_len)

In [ ]:
example = 891
attributions_sum = all_attributions[f"{example}"]
all_tokens2 = all_tokens[f"{example}"]

These functions return which words had the strongest (most positive and most negative) attributions. Change the number of tokens you wish to visualize for your needs. It takes in the attributions and the tokens we grabbed in the previous cell and returns 3 lists: the topk (or bottomk) attributions, their respective token and their position.

Note: Remember that the attributions are with respect to the positive class, so the most impact tokens that helped the model predict the negative class will be in the botk attributed tokens.

In [ ]:
def get_topk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
def get_botk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Convert the values, index of the values, and the token into a pandas Dataframe for visualization. It will be sorted by highest value for attributions to lowest. Alternatively, if youre looking for the most negative attributions, it goes from lowest to highest.

In [ ]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum.cpu(), all_tokens2)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum.cpu(), all_tokens2)

df_high = pd.DataFrame({'Word': top_words_start, 'index':top_word_ind_start, 'attribution': top_words_val_start})

df_low = pd.DataFrame({'Word': bot_words_start, 'index':bot_word_ind_start, 'attribution': bot_words_val_start})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

Here we display our top k positively and negatively attributed tokens for our example.

In [ ]:
df_high['Word'] = df_high['Word'].str.replace('Ġ', '')
df_high

,Word,index,attribution
0,algorithms,720,0.377717
1,algorithms,695,0.280976
2,algorithms,808,0.227178
3,algorithms,704,0.206855
4,search,948,0.161280
5,Search,717,0.153568
6,solving,694,0.147625
7,researchers,688,0.135312
8,algorithm,616,0.135013
9,algorithms,908,0.118958


In [ ]:
df_low['Word'] = df_low['Word'].str.replace('Ġ', '')
df_low

,Word,index,attribution
0,Programming,32,-0.336017
1,Programming,48,-0.268119
2,programs,178,-0.198517
3,Programming,136,-0.176398
4,Computer,68,-0.167329
5,Programming,286,-0.150111
6,programming,229,-0.146878
7,programming,200,-0.125192
8,of,30,-0.117794
9,Logic,47,-0.112482


In [ ]:
d = {"tokens":all_tokens2, "attribution":attributions_sum[:len(all_tokens2)].cpu()}

We notice that there are many repeating tokens in each example that have different positions. While we might want to know how the position plays into the attributions, if we want to know strictly based on the tokens itself, we can add all the duplicate tokens together to get the aggregate attribution for each token. Therefore, we aggregate the attributions strictly based on token type.

In [ ]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False).reset_index()
highest_attrib_tokens['tokens'] = highest_attrib_tokens['tokens'].str.replace('Ġ', '')
highest_attrib_tokens[:10]

,tokens,attribution
0,algorithms,1.350942
1,algorithm,0.269707
2,search,0.161280
3,Search,0.153568
4,researchers,0.135312
5,solving,0.130130
6,agents,0.109145
7,and,0.082687
8,the,0.071416
9,experimental,0.071381


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution']).reset_index()
lowest_attrib_tokens['tokens'] = lowest_attrib_tokens['tokens'].str.replace('Ġ', '')
lowest_attrib_tokens[:10]

,tokens,attribution
0,Programming,-1.018337
1,programming,-0.341622
2,Logic,-0.293834
3,programs,-0.198517
4,Computer,-0.167329
5,of,-0.117236
6,Practice,-0.089079
7,1,-0.085102
8,;,-0.081129
9,counterpart,-0.071546


## Masking the stopwords and non-alpha tokens

There may be some stopwords or punctuations in our top attributed tokens, so now that we have the list of the highest and lowest, we can identify interesting keywords.

In [ ]:
import nltk
from transformers import AutoTokenizer
nltk.download('stopwords')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.append(" ")
stopwords = set(tokenizer2.tokenize(all_stopwords, is_split_into_words =True))
stopwords.update(all_stopwords)
print(stopwords)

{'Ġand', 'do', 'Ġhim', "you've", 'Ġve', 'Ġy', 'Ġwon', 'in', 'Ġhad', 'here', 'all', 'a', 'nor', 'Ġits', 'Ġthen', 'from', 'Ġbecause', 'Ġhimself', 'don', "needn't", 'Ġinto', 'Ġby', 'when', 'there', "'s", 'Ġfrom', 'Ġor', 'his', 'Ġtheirs', 'Ġvery', 'Ġdidn', 'Ġwho', 'Ġcan', 'between', 'Ġdid', "that'll", 'Ġhasn', "she's", 'been', 'Ġi', 'are', 'Ġof', 'on', 'Ġwhom', 've', 'y', "weren't", 'did', 'about', 'no', 'Ġbeing', 'each', 'Ġas', 'Ġwhile', 'Ġit', 'Ġyour', 'Ġain', "'ll", 'Ġall', 'Ġdo', 'some', 'Ġhis', 'isn', 'Ġown', 'Ġin', 'Ġd', "isn't", 'Ġneed', 'now', 'Ġno', 'Ġnow', 'where', 'Ġan', 'most', "mightn't", 'Ġdon', 'Ġmy', 'Ġthe', 'an', 'aren', 'to', 'Ġso', 'Ġshould', 'didn', 'Ġthem', "hadn't", 'have', 'Ġshe', "haven't", 'Ġdown', 'Ġcouldn', 'Ġbetween', 'Ġthey', 'Ġunder', 'Ġwhat', 'out', "you'd", 'doing', 'Ġare', 'during', 'into', 'as', 'theirs', 'through', 'Ġif', 'our', 'their', 'both', 'Ġduring', 'Ġhers', 'Ġm', 'ours', 'ourselves', 'further', "won't", 'before', 'yourself', 'were', 'Ġyou', 'will'

In [ ]:
highest_attrib_tokens[(highest_attrib_tokens['tokens'].str.isalpha()) & ~(highest_attrib_tokens['tokens'].isin(stopwords)) & ~(highest_attrib_tokens['tokens']==0)][:10].reset_index(drop=True)

In [ ]:
lowest_attrib_tokens[(lowest_attrib_tokens['tokens'].str.isalpha()) & ~(lowest_attrib_tokens['tokens'].isin(stopwords)) & ~(lowest_attrib_tokens['tokens']==0)][:10].reset_index(drop=True)

,tokens,attribution
0,Programming,-1.018337
1,programming,-0.341622
2,Logic,-0.293834
3,programs,-0.198517
4,Computer,-0.167329
5,Practice,-0.089079
6,counterpart,-0.071546
7,Set,-0.067201
8,Problems,-0.065523
9,OP,-0.064579


## Aggregate Token Attributions

Using this [notebook](https://colab.research.google.com/drive/1lktilbL1IY4nBanlzCdP8TLsBNfUsl_U?usp=sharing), we can get the files to view the aggregated attributions for the entire dataset for both the positive and negative classes. This means we summed up and averaged the attributions for every instance of any given token throughout the entire dataset (whether or not they have positive or negative attributions).

In [ ]:
df_word = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/longformer_emb_papers.csv")

Here we see the highest attributions for the positive class, meaning that these tokens have the most influence when the model tries to predict positive. All of these words do have relevence to A.I. related topics.

In [ ]:
df_word[:15]

,tokens,attribution
0,learning,0.270925
1,neural,0.159887
2,data,0.128880
3,.,0.108019
4,",",0.092537
5,AI,0.081176
6,training,0.080792
7,dataset,0.071329
8,the,0.067022
9,algorithms,0.058013


We see how the aggregate over the dataset looks without the stopwords and non alpha tokens

In [ ]:
df_word[(df_word['tokens'].str.isalpha()) & ~(df_word['tokens'].isin(stopwords)) & ~(df_word['tokens']==0)][:10].reset_index(drop=True)

,tokens,attribution
0,learning,0.270925
1,neural,0.159887
2,data,0.128880
3,AI,0.081176
4,training,0.080792
5,dataset,0.071329
6,algorithms,0.058013
7,human,0.048780
8,intelligence,0.046269
9,datasets,0.042661


Here we see the largest attributions for the negative class, meaning that these tokens have the most influence when the model predicts negative.

In [ ]:
df_word[:-15:-1].reset_index(drop=True)

,tokens,attribution
0,programming,-0.120735
1,program,-0.088943
2,programs,-0.082175
3,languages,-0.072262
4,language,-0.053761
5,code,-0.052443
6,software,-0.038568
7,compiler,-0.032076
8,Programming,-0.031495
9,syntax,-0.029020


We see how the aggregate over the dataset looks without the stopwords and non alpha tokens

In [ ]:
df_word[(df_word['tokens'].str.isalpha()) & ~(df_word['tokens'].isin(stopwords)) & ~(df_word['tokens']==0)][:-11:-1].reset_index(drop=True)

,tokens,attribution
0,programming,-0.120735
1,program,-0.088943
2,programs,-0.082175
3,languages,-0.072262
4,language,-0.053761
5,code,-0.052443
6,software,-0.038568
7,compiler,-0.032076
8,Programming,-0.031495
9,syntax,-0.029020
